---
title: ChatGPT random generator
description: Насколько хорошо GPT генерирует случайные числа
date: "2025-02-16"
categories: [research]
---

Просто из интереса хочется проверить насколько случайные числа будет генерировать chat GPT если его об этом попросить

Самый простой способ - просить имитировтаь подбрасывание монетки

In [97]:
from openai import OpenAI
import pandas as pd 
import numpy as np
import asyncio
import aiohttp
from tqdm import tqdm
from scipy import stats
import os

In [23]:
TEMPERATURES = np.linspace(0, 2, 10)
client = OpenAI(
    api_key=os.environ.get("OPEN_AI_API_KEY"),
    base_url="https://api.proxyapi.ru/openai/v1", #это провайдер прокси для многих моделек в РФ
)

In [38]:
async def fetch(prompt, temperature):
    async with aiohttp.ClientSession() as session:
        completion = client.chat.completions.create(
                            model="gpt-3.5-turbo",
                            messages=[
                                {"role": "system", "content": "Ты имитируешь игру в Орел или Решка, отвечай только буквами О или Р"},
                                {"role": "user", "content": prompt }], 
        temperature=temperature,)       
        return completion.choices[0].message.content

In [42]:
async def test(prompt, temp, trials):
    results = await asyncio.gather(*(fetch(prompt, temp) for _ in range(trials)))
    return results

## Групповая генерация

Просим в одном промпте подбросить монетку 100 раз и выдать результаты

In [43]:
trials = 5 # по 5 раз перезапускаем результат на одинаковой температуре
group_results = []

for temperature in tqdm(TEMPERATURES):
    res = await test(
        prompt="Подбрось монетку 100 раз и перечисли результаты подбрасывания через запятую", 
        temp=temperature, 
        trials=trials, 
    )
    group_results.append(res)

100%|██████████| 10/10 [08:16<00:00, 49.60s/it]


In [99]:
group_results

[['О, Р, О, О, Р, О, Р, Р, О, О, Р, О, О, Р, Р, О, О, Р, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р, О, Р',
  'О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, О, О, Р, О, Р, 

In [88]:
group_results_metrics = []
columns = [
    "trial", 
    "temperature", 
    "answer_len", 
    "norm_answers_len", 
    "o_share", 
    "p_share", 
    "trash_share", 
]

for idx, trial in enumerate(group_results):
    for series in trial: 
        res = series.upper().split(',')
        
        counts = {'O':0, 'P':0, 'trash':0}
        
        for token in res:
            token = token.strip()
            if token in {'O', 'О',}:
                counts['O'] += 1
            elif token in {'P', 'Р',}:
                counts['P'] += 1
            else:
                counts['trash'] += 1
        
        res_len = len(res)     
        norm_len = counts['O'] + counts['P']
        
        group_results_metrics.append([
            idx, 
            TEMPERATURES[idx], 
            len(res), 
            norm_len, 
            counts['O'] / norm_len if norm_len > 0 else 0, 
            counts['P'] / norm_len if norm_len > 0 else 0, 
            1 - norm_len/res_len,   
        ])
        

In [94]:
tdf = pd.DataFrame(group_results_metrics, columns=columns)

tdf

,trial,temperature,answer_len,norm_answers_len,o_share,p_share,trash_share
0,0,0.000000,109,109,0.522936,0.477064,0.000000
1,0,0.000000,2049,2048,0.600098,0.399902,0.000488
2,0,0.000000,208,208,0.504808,0.495192,0.000000
3,0,0.000000,106,106,0.509434,0.490566,0.000000
4,0,0.000000,2049,2048,0.600098,0.399902,0.000488
5,1,0.222222,113,113,0.610619,0.389381,0.000000
6,1,0.222222,100,100,0.510000,0.490000,0.000000
7,1,0.222222,116,116,0.517241,0.482759,0.000000
8,1,0.222222,108,108,0.518519,0.481481,0.000000
9,1,0.222222,107,107,0.514019,0.485981,0.000000


## Разовая генерация

Просим по очереди генерировать все новые результаты, в духе   
User >>> Подбрось монетку  
Chat >>> O  
User >>> Подбрось монетку  
Chat >>> P   
....  

TBA